# Eva 02

## prepare

In [1]:
from os import path

import torch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import LazyConfig, get_cfg, instantiate
from detectron2.data.detection_utils import read_image
from detectron2.utils.visualizer import ColorMode, Visualizer
from modelscope.hub.snapshot_download import snapshot_download


ModuleNotFoundError: No module named 'detectron2'

In [2]:
base_url = r'/mnt/workspace'
ckpt_base_path = path.join(base_url, 'model')
data_base_path = path.join(base_url, "data")

# replace with your eva02 path 
eva02_modelscope_id = 'zacbi2023/eva02'
eva02_base_path = path.join(ckpt_base_path, eva02_modelscope_id)

# replace with your eva02 config path 
eva02_coco_config_rpath = 'projects/ViTDet/configs/eva2_o365_to_coco/eva2_o365_to_coco_cascade_mask_rcnn_vitdet_l_8attn_1536_lrd0p8.py'
eva02_config_path = path.join(eva02_base_path, eva02_coco_config_rpath)

# replace with your eva02 weights path 
eva02_coco_weights_rpth = 'checkpoints/eva02_L_coco_seg_sys_o365.pth'
eva02_weights_path = path.join(eva02_base_path, eva02_coco_weights_rpth) 

# replace with your image_name
image_name = 'eva_sd_61179.jpg'
input_image_path = path.join(data_base_path, 'image/sd/output/', image_name)
output_image_path = path.join(data_base_path, 'image/eva/output', image_name)

### download model

In [3]:
# ignore semantic segmentation ckpt
if not path.exists(eva02_base_path):
    snapshot_download(eva02_modelscope_id, cache_dir = ckpt_base_path, ignore_file_pattern='eva02_L_ade_seg_upernet_sz640.pth')

### config base

In [3]:
# hardcode threshold
config_file = eva02_config_path
custum_cfg = ['MODEL.RETINANET.SCORE_THRESH_TEST', 0.5,
              'MODEL.ROI_HEADS.SCORE_THRESH_TEST', 0.5,
              'MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH', 0.5,
              'DATASETS.TEST', [],
            'MODEL.WEIGHTS', eva02_weights_path]

### load via LazyConfig

In [4]:
cfg = LazyConfig.load(config_file)
LazyConfig.apply_overrides(cfg, [f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

{'dataloader': {'train': {'dataset': {'names': 'coco_2017_train', '_target_': <function get_detection_dataset_dicts at 0x7f7f7abcf370>}, 'mapper': {'is_train': True, 'augmentations': [{'horizontal': True, '_target_': <class 'detectron2.data.transforms.augmentation_impl.RandomFlip'>}, {'min_scale': 0.1, 'max_scale': 2.0, 'target_height': 1536, 'target_width': 1536, '_target_': <class 'detectron2.data.transforms.augmentation_impl.ResizeScale'>}, {'crop_size': [1536, 1536], 'pad': False, '_target_': <class 'detectron2.data.transforms.augmentation_impl.FixedSizeCrop'>}], 'image_format': 'RGB', 'use_instance_mask': True, '_target_': <class 'detectron2.data.dataset_mapper.DatasetMapper'>, 'recompute_boxes': True}, 'total_batch_size': 64, 'num_workers': 4, '_target_': <function build_detection_train_loader at 0x7f7f7abcf760>}, 'test': {'dataset': {'names': 'coco_2017_val', 'filter_empty': False, '_target_': <function get_detection_dataset_dicts at 0x7f7f7abcf370>}, 'mapper': {'is_train': Fals

### load via get_cfg()

In [6]:
# not use!!!!
# cfg = get_cfg()
# cfg.merge_from_file(config_file)
# cfg.merge_from_list([f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

## Infer

In [5]:
model = instantiate(cfg.model)
model.to('cuda')

GeneralizedRCNN(
  (backbone): SimpleFeaturePyramid(
    (simfp_2): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): LayerNorm()
      (2): GELU(approximate='none')
      (3): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
      (4): Conv2d(
        256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (5): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_3): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): Conv2d(
        512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (2): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_4): Sequential(
      (0): Conv2d(
        1024, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
DetectionCheckpointer(model).load(eva02_weights_path)
model.eval()

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


GeneralizedRCNN(
  (backbone): SimpleFeaturePyramid(
    (simfp_2): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): LayerNorm()
      (2): GELU(approximate='none')
      (3): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
      (4): Conv2d(
        256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (5): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_3): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): Conv2d(
        512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (2): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_4): Sequential(
      (0): Conv2d(
        1024, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
original_image = read_image(input_image_path, format="BGR")
height, width = original_image.shape[:2]
image = torch.as_tensor(original_image.astype("float16").transpose(2, 0, 1))
inputs = {"image": image, "height": height, "width": width}
predictions = model([inputs])[0]

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.06 GiB. GPU 0 has a total capacty of 22.20 GiB of which 4.68 GiB is free. Process 31818 has 17.52 GiB memory in use. Of the allocated memory 12.14 GiB is allocated by PyTorch, and 5.10 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
original_image = original_image[:, :, ::-1]

NameError: name 'original_image' is not defined

In [11]:
# 可视化器
visualizer = Visualizer(original_image, None,
                                instance_mode=ColorMode.IMAGE)

In [12]:
cpu_device = torch.device("cpu")
instances = predictions["instances"].to(cpu_device)

In [13]:
# 可视化输出
vis_output = visualizer.draw_instance_predictions(
                    predictions=instances)

In [14]:
# save
vis_output.save(output_image_path)

In [18]:
instances = predictions['instances']
instances

Instances(num_instances=4, image_height=768, image_width=768, fields=[pred_boxes: Boxes(tensor([[2.0944e+02, 5.2295e+00, 5.7464e+02, 7.6118e+02],
        [1.0083e+02, 3.3497e+02, 5.1348e+02, 6.7934e+02],
        [1.0726e-01, 2.2810e+01, 4.3642e+01, 3.1540e+02],
        [2.0703e+02, 5.0231e+00, 5.7440e+02, 7.6125e+02]], device='cuda:0',
       grad_fn=<IndexBackward0>)), scores: tensor([0.9957, 0.9942, 0.0803, 0.0531], device='cuda:0',
       grad_fn=<IndexBackward0>), pred_classes: tensor([ 0, 15, 26, 15], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False

In [26]:
pred_masks = instances.pred_masks
pred_classes = instances.pred_classes

In [29]:
torch.sum(pred_masks[1]) + torch.sum(pred_masks[3]), torch.sum(instances.pred_masks[instances.pred_classes == 15])

(tensor(234792, device='cuda:0'), tensor(234792, device='cuda:0'))